In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Решение нелинейной краевой задачи для уравнения второго порядка методами стрельбы и квазилинеаризации

Получим численное решение следующей нелинейной краевой задачи.

#### $y'' + \frac{0,5}{1-0,5y}(y')^2=0, 0 < x <= 1$

#### $y(0)=1.5, y(1)=0$

### Общая постановка задачи

##### $y''=f(x,y,y'), 0<x<=1,$
##### $y(0)=Y_{0}, y(1)=Y_{1}$

In [ ]:
Y0 = 1.5
Y1 = 0

### Метод стрельбы

Метод стрельбы для решения краевой задачи базируется на том, что имеются удобные способы численного решения задачи Коши, т. е. задачи

##### $y''=f(x,y,y'), 0<x<=1$
##### $y(0)=Y_{0}, y'(0)=\alpha$

В нашем случае: 
##### $y'_{1}(x)=y_2(x)$
##### $y'_{2}(x)=\frac{(0.5y_{2}'(x))^2}{0.5y_{1}(x)-1}$
##### $y_{1}(0)=Y_{0}, y_{2}(0)=\alpha$

где $Y_{0}$ — ордината точки $(0, Y_{0})$, из которой выходит интегральная кривая; $\alpha$ — пристрелочный параметр, можно считать, что он определяет угол наклона интегральной кривой к оси $x$ при выходе из точки $(0, Y_{0})$. При фиксированном $Y_{0}$ решение задачи Коши будет зависеть от пристрелочного параметра, $y=y(x,\alpha)$. В дальнейшем будем считать такую зависимость непрерывной, хотя это не всегда так. При $x=1$ решение $y(x,\alpha)$ зависит только от $\alpha$:

##### $y(x,\alpha)|_{x=1}=y(1,\alpha)$

Найдём $\alpha$ такой, что $y(1,\alpha)=Y_{1}$. Задача свелась к решению уравнения:

$F(\alpha)=0, F(\alpha)=y(1,\alpha)-Y_{1}$

В случае использования для решения уравнения $F(\alpha)=0$ метода Ньютона очевидная реализация опирается на приемы, связанные с численным дифференцированием. Например, задаем $\alpha$ а затем последующие $\alpha_{n}$ вычисляем по рекуррентной формуле:

#### $\alpha_{n+1}=\alpha_{n} - \frac{F(\alpha_{n})}{F'(\alpha_{n})}, n=0,1,...$

#### $F'(\alpha_{n})=\frac{F(\alpha_{n} + h) - F(\alpha_{n})}{h}$

In [ ]:
def graph(x, y, filename, title):
    """
    Build graph
    """
    
    plt.figure(figsize=[10, 4])
    plt.plot(x, y, 'b-')
    
    plt.title(title)
    plt.grid()
    plt.xlabel('x')
    plt.ylabel('y')

    plt.savefig(filename)

In [ ]:
def f(x,y):
    """
    ODE's right side
    """
    return np.array([y[1],-(0.5 * y[1] * y[1])/(1 - 0.5 * y[0])])

In [ ]:
def rungekutta_iter(y, x, tau, idx):
    k1 = tau * f(x[idx], y[idx])
    k2 = tau * f(x[idx] + tau/2, y[idx] + k1/2)
    k3 = tau * f(x[idx] + tau, y[idx] - k1 + 2 * k2)
    return y[idx] + (k1 + 4 * k2 + k3) / 6

def solve_rungekutta(f,y0,x):
    """
    Solving system of ODE with Runge-Kutta method
    """
    n = len(x)
    tau = x[1] - x[0]
    y = np.zeros((n, len(y0)))
    
    y[0] = y0

    for idx in range(n-1):
        y[idx+1] = rungekutta_iter(y, x, tau, idx)

    return y

def solve_сauchy(f, Y0, alpha):
    """
    Solve Сauchy problem
    """
    y0 = np.array([Y0, alpha])

    # 0 < x <= 1
    X = 1
    # Number of points on (0,X] period
    points_num = 1001
    x = np.linspace(0, X, points_num)

    # Solve ODE
    sol = solve_rungekutta(f, y0, x)
    y = sol[:,0]
    
    # y(1,alpha)
    y_1 = y[-1]
    
    return x, sol[:,0], y_1

def solve_shooting(f, Y0, Y1, eps, alpha):
    """
    Solve problem using Shooting method
    """
    # Numerical derivation step
    h = 1e-5

    x = []
    y = []
    
    while(True):

        # Find y(1, alpha) and y(1, alpha + h)
        x, y, y_1_alpha = solve_сauchy(f, Y0, alpha)
        _, _, y_1_alpha_h = solve_сauchy(f, Y0, alpha + h)

        # F(alpha_n)
        F_alpha_n = y_1_alpha - Y1 

        # Stop if needed precision is achieved
        if abs(F_alpha_n) < eps:
            break
        
        # F(alpha_n + h)
        F_alpha_n_h = y_1_alpha_h - Y1

        # Numerical derivative
        F_alpha_n_der = (F_alpha_n_h - F_alpha_n) / h

        # Next alpha
        alpha = alpha - F_alpha_n / F_alpha_n_der

    return x, y

In [ ]:
x, y = solve_shooting(f, Y0, Y1, 1e-6, 0.1)
graph(x, y, f"pictures/shooting.png", f"Shooting method")

<img src="pictures/shooting.png" alt="shooting"/>

### Метод квазилинеаризации

Метод Ньютона сводит решение нелинейной краевой задачи к решению серии линейных краевых задач и состоит в следующем.
Пусть для нелинейной краевой задачи известна функция $y_{0}(x)$ удовлетворяющая граничным условиям и грубо приближенно равная искомому $y(x)$. 
Положим $y(x)=y_{0}(x)+v(x)$, где $v(x)$ - поправка к нулевому приближению к $y_{0}(x)$. Подставим выражение для $y(x)$ в изначальное уравнение:
$$y''(x)=y_{0}''(x)+v''(x)$$
$$f(x,y_{0}+v,y_{0}'+v')=f(x,y_{0},y_{0}')+\frac{\delta f(x,y_{0},y_{0}')}{\delta y}v+\frac{\delta f(x,y_{0},y_{0}')}{\delta y'}v'+O(v^2+|v'|^2)$$
Отбрасывая остаточный член $O(v^2+|v'|^2)$ получим линейную краевую задачу для нахождения поправки $\tilde v(x)$ с нулевыми краевыми условиями:

$$\tilde v''=p(x)\tilde v'+g(x)\tilde v+r(x)$$
$$\tilde v(0)=0, \tilde v(1)=0$$

$$p(x)=\frac{\delta f(x,y_{0},y_{0}')}{\delta y'}, q(x)=\frac{\delta f(x,y_{0},y_{0}')}{\delta y}, r(x)=f(x,y_{0},y_{0}')-y_{0}''$$

Мы получили уравнение в вариациях с невязкой нулевого решения в качестве правой части. Решая линейную краевую задачу численным методом, найдем поправку $\tilde v$ и примем за первое приближение $y_{1}(x)=y_{0}(x)+\tilde v$.

Аналогично положим $y(x)=y_{1}(x)+\tilde v_{1}$ и найдем следующее приближение. Для него будем решать то же уравнение в вариациях, но с коэффициентами, вычисленными по первому приближению, и с правой частью - невязкой первого приближения. Продолжая процесс до тех пор, пока не будут выполнены неравенства $max|\tilde v(x)|<=\epsilon, 0<=x<=1$, где $\epsilon$ - требуемая точность.

Рассмотрим нашу задачу и применим к ней метод квазилинеализации.

$$f(y^{m+1},y'^{m+1})=f(y^{m},y'^{m}) + \frac{\delta f}{\delta y}(y^{m},y'^{m})(y^{m+1}-y^{m})$$

$$f(y,y')=\frac{0.5y'^2}{0.5y-1}, \frac{\delta f}{\delta y}(y^m,y'^m)=-0.5(\frac{y'^m}{0.5y^m-1})^2$$

$$\frac{y_{k+1}^{m+1} - 2y_{k}^{m+1} + y_{k-1}^{m+1}}{h^2}=\frac{0.5((y')_{k}^{m+1})^2}{0.5y_{k}^{m+1}-1}$$

$$y_{0}^{m+1}=y(0)=Y_{0}, y_{N}^{m+1}=y(1)=0$$

$$\frac{y_{k+1}^{m+1} - 2y_{k}^{m+1} + y_{k-1}^{m+1}}{h^2}=\frac{0.5((y')_{k}^{m})^2}{0.5y_{k}^{m}-1} - \frac{0.25((y')_{k}^{m})^2}{(0.5y_{k}^{m}-1)^2}(y_{k}^{m+1}-y_{k}^{m})$$

###### $$\frac{y_{k+1}^{m+1} - 2y_{k}^{m+1} + y_{k-1}^{m+1}}{h^2}-\frac{0.5}{0.5y_{k}^{m}-1}(\frac{3y_{k+1}^{m}-4y_{k}^{m}+y_{k-1}^{m}}{2h})^2 + \frac{0.25}{(0.5y_{k}^{m}-1)^2}(\frac{3y_{k+1}^{m}-4y_{k}^{m}+y_{k-1}^{m}}{2h})^2(y_{k}^{m+1}-y_{k}^{m})=0$$

In [ ]:
def quasilin_iter_get_matrix(y, h, N):
    """
    Build matrix of the system
    """
    M = np.zeros((N + 1, N + 1))

    for i in range(N + 1):
        for j in range(N+1):

            # y_{0}^{m+1} coeff
            if i == 0:
                if j == 0:
                    M[i][j] = 1
                else:
                    M[i][j] = 0
                continue

            # y_{N}^{m+1} coeff
            if i == N:
                if j == N:
                    M[i][j] = 1
                else:
                    M[i][j] = 0
                continue
            
            # y_{k}^{m+1} coeff
            if i == j:
                deriv_sqr = pow((3 * y[i+1] - 4 * y[i] + y[i-1]) / (2 * h), 2)
                frac = 0.5 / (0.5 * y[i] - 1)
                M[i][j] = - 2 / pow(h, 2) + pow(frac, 2) * deriv_sqr

            # y_{k-1}^{m+1} coeff
            elif i - 1 == j:
                M[i][j] = 1 / pow(h,2)

            # y_{k+1}^{m+1} coeff
            elif i + 1 == j:
                M[i][j] = 1 / pow(h,2)
            
            else:
                M[i][j] = 0

    return M

def quasilin_iter_get_right_side(y, h, N, Y0, Y1):
    """
    Build right side of the system
    """
    right_side = np.zeros(N + 1)
    right_side[0] = Y0
    right_side[N] = Y1

    for i in range(1,N):
        deriv_sqr = pow((3 * y[i+1] - 4 * y[i] + y[i-1]) / (2 * h), 2)
        frac = 0.5 / (0.5 * y[i] - 1)
        right_side[i] = frac * deriv_sqr + pow(frac, 2) * deriv_sqr * y[i]

    return right_side

def quasilin_iter(y, h, N, Y0, Y1):
    """
    Iterate once using common algorithm
    """
    M = quasilin_iter_get_matrix(y, h, N)
    right_side = quasilin_iter_get_right_side(y, h, N, Y0, Y1)

    return np.linalg.solve(M, right_side)
    
def solve_quasilin(Y0, Y1, eps, h, iteration_num):
    """
    Iterate till the desired accuracy is succeded
    """
    
    # Number of points on the period [0;1]
    points_num = int(1/h)

    # First approximation
    y_prev = np.full(points_num + 1, 1)

    # Current iteration computation result
    y_cur  = np.zeros(points_num + 1)

    for iteration_idx in range(iteration_num):

        # Compute current accuracy
        y_cur = quasilin_iter(y_prev, h, points_num, Y0, Y1)
        
        # If desired accuracy is succeded, stop
        if np.max(np.abs(y_cur - y_prev)) < eps:
            break

        # Move to the next iteration
        y_prev = y_cur

    x = np.array([i * h for i in range(points_num+1)])
    return x, y_cur


In [ ]:
x, y = solve_quasilin(Y0, Y1, 1e-3, 0.01, 100)
graph(x, y, f"pictures/quasilin.png", f"Quasilinearisation method")

<img src="pictures/quasilin.png" alt="quasilin"/>